In [238]:
## Coursework 2: Museum visits and Funding: Cleaning Museum Visits data, join against Funding data.
# Import modules, read in data

# assumes that the data is in the same folder as this notebook
data_filename = 'July_to_September_2020.csv'
data_filename2 = 'museum_funding_1.csv'
output_filename = 'funding_visits.csv'

# Importing all the packages
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sms
import seaborn as sn
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


# Read in Museum Data
mvisits = pd.read_csv(data_filename, header = 0, encoding = 'unicode_escape')
funding = pd.read_csv(data_filename2, header = 0)

In [239]:
# Data Exploration
mvisits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11269 entries, 0 to 11268
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   museum  11269 non-null  object 
 1   year    11269 non-null  int64  
 2   month   11269 non-null  int64  
 3   visits  11269 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 352.3+ KB


In [240]:
mvisits.head()

,museum,year,month,visits
0,TOTAL VISITOR FIGURES,2004,4,0.0
1,TOTAL VISITOR FIGURES,2004,5,0.0
2,TOTAL VISITOR FIGURES,2004,6,0.0
3,TOTAL VISITOR FIGURES,2004,7,0.0
4,TOTAL VISITOR FIGURES,2004,8,0.0


In [241]:
# creating a column to capture all rows that have a "Total" in the museum field
# pull unique rows to validate that all of these rows should be removed
mvisits['Total'] = mvisits.museum.str.contains('TOTAL', regex = False)
mvisits[(mvisits['Total'] == True)]['museum'].unique()

array(['TOTAL VISITOR FIGURES', 'IMPERIAL WAR MUSEUM (IWM) TOTAL',
       'NATURAL HISTORY MUSEUM (NHM) TOTAL',
       'SCIENCE MUSEUM GROUP (SMG) TOTAL', 'ROYAL ARMOURIES (RA) TOTAL',
       'TATE GALLERIES TOTAL', 'VICTORIA AND ALBERT MUSUEM (V&A) TOTAL',
       'TYNE & WEAR TOTAL'], dtype=object)

In [242]:
# Drop these rows from the dataset as they are totals
mvisits.drop(mvisits[(mvisits['Total'] == True)].index, axis=0, inplace=True)
# validate the number of rows have decreased
mvisits.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9703 entries, 204 to 11268
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   museum  9703 non-null   object 
 1   year    9703 non-null   int64  
 2   month   9703 non-null   int64  
 3   visits  9703 non-null   float64
 4   Total   9703 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 388.5+ KB


In [243]:
# restricting to data that is 2008 and later
mvisits = mvisits[mvisits['year'] >= 2008]
mvisits.head()

,museum,year,month,visits,Total
249,BRITISH MUSEUM,2008,1,546448.0,False
250,BRITISH MUSEUM,2008,2,580093.0,False
251,BRITISH MUSEUM,2008,3,635357.0,False
252,BRITISH MUSEUM,2008,4,455743.0,False
253,BRITISH MUSEUM,2008,5,454604.0,False


In [244]:
# for loop to add fiscal year data to dataset
mvisits['fiscal_year'] = mvisits['year'] 
j = 1 # personally don't know how to do the loop without using a counter

# for all rows in mvisits dataset, if month is less than or equal to 3,
# set fiscal year = 'year'-1 / 'year'
# else, set fiscal year = 'year' / 'year' + 1

for row in range(0, len(mvisits)-1):
    if mvisits.iloc[j,2] <= 3:
        mvisits.iloc[j,5] = str(f"{mvisits.iloc[j,1]-1}/{mvisits.iloc[j,1]}")
        j = j + 1
    else:
        mvisits.iloc[j,5] = str(f"{mvisits.iloc[j,1]}/{mvisits.iloc[j,1]+1}")
        j = j + 1

In [245]:
# spot checking dataset to ensure everything populated correctly
mvisits.sample(6, random_state=42)

,museum,year,month,visits,Total,fiscal_year
8944,(T&W) GREAT NORTH MUSEUM,2018,7,0.0,False,2018/2019
3641,(SMG) NATIONAL MEDIA MUSEUM,2013,6,42243.0,False,2013/2014
1901,(IWM) NORTH,2014,9,30700.0,False,2014/2015
10236,(T&W) HATTON GALLERY,2014,3,3714.0,False,2013/2014
5477,(RA) FORT NELSON,2020,9,7023.0,False,2020/2021
3038,NATIONAL MUSEUMS LIVERPOOL,2011,12,159971.0,False,2011/2012


In [246]:
funding['organisation'].unique()

array(['British Museum', 'Geffrye Museum', 'Horniman Museum',
       'Imperial War Museum', 'National Gallery',
       'National Museums Liverpool', 'Science Museum Group',
       'National Portrait Gallery', 'Natural History Museum',
       'Royal Armouries', 'Royal Museums Greenwich',
       "Sir John Soane's Museum", 'Tate Gallery',
       'Victoria and Albert Museum ', 'Wallace Collection',
       'English Heritage Trust', 'Historic England', 'British Library',
       'Arts Council England', 'British Film Institute'], dtype=object)

In [247]:
mvisits['museum'].unique()

array(['BRITISH MUSEUM', 'MUSEUM OF THE HOME',
       'HORNIMAN MUSEUM (Excluding visits to the Garden)', '(IWM) LONDON',
       '(IWM) HMS BELFAST', '(IWM) CHURCHILL WAR ROOMS', '(IWM) DUXFORD',
       '(IWM) NORTH', 'NATIONAL GALLERY', '(NHM) SOUTH KENSINGTON',
       '(NHM) TRING', 'ROYAL MUSEUMS GREENWICH',
       'NATIONAL MUSEUMS LIVERPOOL', '(SMG) SOUTH KENSINGTON',
       '(SMG) NATIONAL MEDIA MUSEUM', '(SMG) NATIONAL RAILWAY MUSEUM',
       '(SMG) LOCOMOTION AT SHILDON',
       '(SMG) MUSEUM OF SCIENCE AND INDUSTRY, MANCHESTER',
       'NATIONAL COAL MINING MUSEUM FOR ENGLAND',
       '(SMG) SWINDON (WROUGHTON)', 'NATIONAL PORTRAIT GALLERY',
       '(RA) LEEDS', '(RA) FORT NELSON',
       '(RA) WHITE TOWER (BASED AT THE TOWER OF LONDON)',
       "SIR JOHN SOANE'S MUSEUM", 'TATE BRITAIN', 'TATE MODERN',
       'TATE LIVERPOOL', 'TATE ST IVES', '(V&A) SOUTH KENSINGTON',
       '(V&A) MUSEUM OF CHILDHOOD, BETHNAL GREEN',
       '(V&A) THEATRE MUSEUM, COVENT GARDEN', '(V&A) BLYTHE

In [248]:
# grouping is determined in text names, except for:
# Museum visit data: 'Visit figures for the National Coal Mining Museum are no longer collated in 
#       the Science Museum Group total, but added to the DCMS total separately.' therefore, National Coal Mining 
#       will be considered as part of SMG given that it is an analysis over the past ten years. We are assuming that this
#       is a recent change
# DCMS performance indicators: 'As of April 2015, Tyne and Wear museums are no longer sponsored by DCMS and have therefore been excluded from DCMS total visits for all years.'
#       therefore, T&W will be listed as 'Other' and should be dropped from the data.
# Museum of London/Docklands: based on website, appears to be sponsored by Arts Council England, so I've assigned them to this group, but probably should be dropped.

museumgroups = {'BRITISH MUSEUM': 'British Museum',
                'MUSEUM OF THE HOME': 'Geffrye Museum',
                'HORNIMAN MUSEUM (Excluding visits to the Garden)': 'Horniman Museum',
                '(IWM) LONDON' : 'Imperial War Museum',
                '(IWM) HMS BELFAST' : 'Imperial War Museum',
                '(IWM) CHURCHILL WAR ROOMS' : 'Imperial War Museum',
                '(IWM) DUXFORD' : 'Imperial War Museum',
                '(IWM) NORTH' : 'Imperial War Museum',
                'NATIONAL GALLERY': 'National Gallery',
                '(NHM) SOUTH KENSINGTON' : 'Natural History Museum',
                '(NHM) TRING' : 'Natural History Museum',
                'ROYAL MUSEUMS GREENWICH' : 'Royal Museums Greenwich',
                'NATIONAL MUSEUMS LIVERPOOL' : 'National Museums Liverpool',
                '(SMG) SOUTH KENSINGTON' : 'Science Museum Group',
                '(SMG) NATIONAL MEDIA MUSEUM' : 'Science Museum Group',
                '(SMG) NATIONAL RAILWAY MUSEUM' : 'Science Museum Group',
                '(SMG) LOCOMOTION AT SHILDON' : 'Science Museum Group',
                '(SMG) MUSEUM OF SCIENCE AND INDUSTRY, MANCHESTER' : 'Science Museum Group',
                'NATIONAL COAL MINING MUSEUM FOR ENGLAND' :  'Science Museum Group', 
                '(SMG) SWINDON (WROUGHTON)' : 'Science Museum Group',
                'NATIONAL PORTRAIT GALLERY' : 'National Portrait Gallery',
                '(RA) LEEDS' : 'Royal Armouries',
                '(RA) FORT NELSON' : 'Royal Armouries',
                '(RA) WHITE TOWER (BASED AT THE TOWER OF LONDON)' : 'Royal Armouries',
                "SIR JOHN SOANE'S MUSEUM" : "Sir John Soane's Museum",
                'TATE BRITAIN' : 'Tate Gallery',
                'TATE MODERN' : 'Tate Gallery',
                'TATE LIVERPOOL' : 'Tate Gallery',
                'TATE ST IVES' : 'Tate Gallery',
                '(V&A) SOUTH KENSINGTON' : 'Victoria and Albert Museum ',
                '(V&A) MUSEUM OF CHILDHOOD, BETHNAL GREEN' : 'Victoria and Albert Museum ',
                '(V&A) THEATRE MUSEUM, COVENT GARDEN' : 'Victoria and Albert Museum ',
                '(V&A) BLYTHE HOUSE' : 'Victoria and Albert Museum ',
                'WALLACE COLLECTION' : 'Wallace Collection',
                '(T&W) ARBEIA' : 'Other',
                '(T&W) DISCOVERY' : 'Other',
                '(T&W) HANCOCK' : 'Other',
                '(T&W) GREAT NORTH MUSEUM' : 'Other',
                '(T&W) LAING' : 'Other',
                '(T&W) MONKWEARMOUTH' : 'Other',
                '(T&W) WASHINGTON F PIT' : 'Other',
                '(T&W) SEGEDUNUM' : 'Other',
                '(T&W) SHIPLEY' : 'Other',
                '(T&W) SOUTH SHIELDS' : 'Other',
                '(T&W) HATTON GALLERY' : 'Other',
                '(T&W) STEPHENSON' : 'Other',
                '(T&W) SUNDERLAND' : 'Other',
                '(T&W)\xa0MUSEUM STORE\xa0(SHARED BY T&W AND BEAMISH MUSEUM)' : 'Other',
                'MUSEUM OF LONDON' : 'Arts Council England',
                'MUSEUM IN DOCKLANDS': 'Arts Council England'}

print(len(museumgroups.keys()))

50


In [249]:
mvisitsgroup = mvisits.set_index('museum').groupby([museumgroups,'fiscal_year']).agg({'visits':'sum'}).reset_index()
# checking a sample of the data, have validated it via excel pivot tables
mvisitsgroup.sample(6, random_state=42)

,level_0,fiscal_year,visits
115,Natural History Museum,2010/2011,4809647.0
15,British Museum,2008/2009,5472056.0
211,Victoria and Albert Museum,2009/2010,2746050.0
126,Other,2007/2008,307647.0
6,Arts Council England,2013/2014,0.0
170,Science Museum Group,2010/2011,4201045.0


In [250]:
# fix column names
cols = mvisitsgroup.columns.values
cols[0]  = 'organisation'
cols[1]  = 'fiscal_year'
cols[2]  = 'visits'
mvisitsgroup.columns = cols

In [251]:
mvisitsgroup.head()

,organisation,fiscal_year,visits
0,Arts Council England,2007/2008,112583.0
1,Arts Council England,2008/2009,0.0
2,Arts Council England,2009/2010,0.0
3,Arts Council England,2010/2011,0.0
4,Arts Council England,2011/2012,0.0


In [252]:
# left join funding data with visits data
funding_visits = pd.merge(funding, mvisitsgroup, how = 'left', left_on = ['organisation', 'year'], right_on = ['organisation', 'fiscal_year'])
funding_visits.sample(10, random_state=42)

,organisation,year,total_income,fundraising_income,amount_gia,value_donated_objects,ratio_fundraising_to_gia,fiscal_year,visits
132,Tate Gallery,2014/2015,209362000.0,130383000.0,32149000.0,72742000.0,4.06,2014/2015,7904121.0
148,Natural History Museum,2015/2016,76171000.0,6187000.0,42065000.0,294000.0,0.15,2015/2016,5351797.0
93,Victoria and Albert Museum,2012/2013,81930000.0,12378000.0,48205000.0,1214000.0,0.26,2012/2013,3734257.0
180,British Museum,2017/2018,104425000.0,23300000.0,53573000.0,1125000.0,0.43,2017/2018,5822515.0
15,English Heritage Trust,2008/2009,177443000.0,18777183.0,129358000.0,NaN,0.15,NaN,NaN
115,English Heritage Trust,2013/2014,186546000.0,30049290.0,99846000.0,5000.0,0.30,NaN,NaN
172,Tate Gallery,2016/2017,162295000.0,69777000.0,40251000.0,20995000.0,1.73,2016/2017,8448220.0
209,Royal Armouries,2018/2019,24053000.0,1277882.0,9461000.0,576000.0,0.14,2018/2019,2294347.0
75,English Heritage Trust,2011/2012,175786208.0,21528676.0,121193000.0,12950.0,0.18,NaN,NaN
142,Horniman Museum,2015/2016,6841000.0,408944.0,3881000.0,12000.0,0.11,2015/2016,793883.0


In [253]:
# dropping nulls as these organisations ('English Heritage Trust', 'Historic England', 'British Library', 'British Film Institute') are not in visits data, so will drive false nulls

funding_visits.drop(funding_visits[funding_visits['fiscal_year'].isna()].index, axis=0, inplace=True)
funding_visits.sample(10, random_state=42)

,organisation,year,total_income,fundraising_income,amount_gia,value_donated_objects,ratio_fundraising_to_gia,fiscal_year,visits
23,Imperial War Museum,2009/2010,46407000.0,5881000.0,24163000.0,441000.0,0.24,2009/2010,2079264.00
53,Victoria and Albert Museum,2010/2011,75748000.0,12517000.0,44318000.0,4965000.0,0.28,2010/2011,3048887.00
171,Sir John Soane's Museum,2016/2017,3803647.0,962614.0,1983000.0,NaN,0.49,2016/2017,121504.00
34,Wallace Collection,2009/2010,7939000.0,2132000.0,4301000.0,NaN,0.50,2009/2010,382772.64
83,Imperial War Museum,2012/2013,50116000.0,3432000.0,31186000.0,55000.0,0.11,2012/2013,1990343.00
20,British Museum,2009/2010,85575000.0,13555000.0,48348000.0,2218000.0,0.28,2009/2010,5650388.00
147,National Portrait Gallery,2015/2016,22865000.0,7919000.0,6634000.0,4161000.0,1.19,2015/2016,2102975.00
212,Tate Gallery,2018/2019,142003000.0,44912000.0,37566000.0,10805000.0,1.20,2018/2019,8157755.00
133,Victoria and Albert Museum,2014/2015,101436000.0,46077000.0,31146000.0,16878000.0,1.48,2014/2015,3736204.00
172,Tate Gallery,2016/2017,162295000.0,69777000.0,40251000.0,20995000.0,1.73,2016/2017,8448220.00


In [254]:
funding_visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 218
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   organisation              176 non-null    object 
 1   year                      176 non-null    object 
 2   total_income              176 non-null    float64
 3   fundraising_income        176 non-null    float64
 4   amount_gia                176 non-null    float64
 5   value_donated_objects     136 non-null    float64
 6   ratio_fundraising_to_gia  176 non-null    float64
 7   fiscal_year               176 non-null    object 
 8   visits                    176 non-null    float64
dtypes: float64(6), object(3)
memory usage: 13.8+ KB


In [255]:
funding_visits.to_csv(output_filename)